# *Mount to google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/GovHackTrafficDeakin/gov hack nat

/content/drive/MyDrive/Colab Notebooks/GovHackTrafficDeakin/gov hack nat


# *Turn geojson files into csv
* = optional

In [ ]:
import geopandas as gpd

def read_geojson_file(file_path):
    return gpd.read_file(file_path)

def geojson_to_csv(geojson_data, csv_file_path):
    # Extract the necessary attributes and location
    geojson_data['latitude'] = geojson_data.geometry.y
    geojson_data['longitude'] = geojson_data.geometry.x
    # Drop the geometry column as it's no longer needed
    geojson_data = geojson_data.drop(columns='geometry')
    # Save to CSV
    geojson_data.to_csv(csv_file_path, index=False)

file_path = "Traffic_Lights.geojson"
csv_file_path = "Traffic_Lights_csv.csv"
geojson_data = read_geojson_file(file_path)
geojson_to_csv(geojson_data, csv_file_path)

# NODE.csv (Count num of accidents cases in a location)

In [100]:
import pandas as pd
# Function to round LATITUDE and LONGITUDE to n decimal places, same as other dfs
def round_lat_lon(df, places=5):
    df['LATITUDE'] = df['LATITUDE'].round(places)
    df['LONGITUDE'] = df['LONGITUDE'].round(places)


In [101]:
node_df = pd.read_csv('NODE.csv')
round_lat_lon(node_df, 4)

node_df.head()

,ACCIDENT_NO,NODE_ID,NODE_TYPE,AMG_X,AMG_Y,LGA_NAME,LGA NAME ALL,DEG_URBAN_NAME,LATITUDE,LONGITUDE,POSTCODE_CRASH
0,T20120000009,249102,N,2563628.962,2362700.434,BAW BAW,BAW BAW,RURAL_VICTORIA,-38.2350,145.7267,3981
1,T20120000012,41780,N,2514194.683,2399718.586,MONASH,MONASH,MELB_URBAN,-37.9035,145.1614,3170
2,T20120000013,69811,I,2508689.410,2396130.003,KINGSTON,KINGSTON,MELB_URBAN,-37.9359,145.0988,3169
3,T20120000018,22636,I,2227360.371,2804744.001,MILDURA,MILDURA,RURAL_VICTORIA,-34.2159,142.0439,3505
4,T20120000021,248597,N,2480590.789,2347977.747,MORNINGTON PENINSULA,MORNINGTON PENINSULA,MELB_URBAN,-38.3696,144.7779,3942


# *check for duplicates in LATITUDE and LONGITUDE

In [ ]:
# Group the data by NODE_ID and check for duplicates in LATITUDE and LONGITUDE
duplicates_df = node_df.groupby('NODE_ID').agg({
    'LATITUDE': lambda x: len(x.unique()) > 1,
    'LONGITUDE': lambda x: len(x.unique()) > 1,
    'POSTCODE_CRASH': lambda x: len(x.unique()) > 1
}).any(axis=1)

# Print the result
print(duplicates_df[duplicates_df])

# If the result is empty, it means that NODE_ID always has the same LATITUDE and LONGITUDE
# If there are rows in the result, it means that there are instances where the same NODE_ID has different LATITUDE and LONGITUDE values


Series([], dtype: bool)


# Count num of accidents cases in a location, NODE.csv, Make num of incidents new column

In [102]:
node_df['NUM_INCIDENTS'] = node_df.groupby('NODE_ID')['ACCIDENT_NO'].transform('count')

node_df.head()

,ACCIDENT_NO,NODE_ID,NODE_TYPE,AMG_X,AMG_Y,LGA_NAME,LGA NAME ALL,DEG_URBAN_NAME,LATITUDE,LONGITUDE,POSTCODE_CRASH,NUM_INCIDENTS
0,T20120000009,249102,N,2563628.962,2362700.434,BAW BAW,BAW BAW,RURAL_VICTORIA,-38.2350,145.7267,3981,1
1,T20120000012,41780,N,2514194.683,2399718.586,MONASH,MONASH,MELB_URBAN,-37.9035,145.1614,3170,14
2,T20120000013,69811,I,2508689.410,2396130.003,KINGSTON,KINGSTON,MELB_URBAN,-37.9359,145.0988,3169,1
3,T20120000018,22636,I,2227360.371,2804744.001,MILDURA,MILDURA,RURAL_VICTORIA,-34.2159,142.0439,3505,2
4,T20120000021,248597,N,2480590.789,2347977.747,MORNINGTON PENINSULA,MORNINGTON PENINSULA,MELB_URBAN,-38.3696,144.7779,3942,1


# *Check accident_event

In [ ]:
accevt_df = pd.read_csv('ACCIDENT_EVENT.csv')

accevt_df.head()

,ACCIDENT_NO,EVENT_SEQ_NO,EVENT_TYPE,EVENT_TYPE_DESC,VEHICLE_1_ID,VEHICLE_1_COLL_PT,VEHICLE 1 COLL PT DESC,VEHICLE_2_ID,VEHICLE_2_COLL_PT,VEHICLE 2 COLL PT DESC,PERSON_ID,OBJECT_TYPE,OBJECT_TYPE_DESC
0,T20120003941,1,3,Ran off carriageway,A,9,Not known or Not Applicable,NaN,NaN,NaN,NaN,99,Not Applicable
1,T20120003941,2,C,Collision,A,5,Left front corner,NaN,NaN,NaN,NaN,7,Guard Rail
2,T20120003941,3,C,Collision,A,9,Not known or Not Applicable,B,8,Left rear corner,NaN,99,Not Applicable
3,T20120003941,4,2,Fell from vehicle,A,9,Not known or Not Applicable,NaN,NaN,NaN,NaN,99,Not Applicable
4,T20120003958,1,1,Rollover on/off carriageway,A,1,Right front corner,NaN,NaN,NaN,NaN,99,Not Applicable


In [ ]:
duplicate_entries = accevt_df['ACCIDENT_NO'].duplicated()
num_duplicate_entries = duplicate_entries.sum()
print("Number of duplicate entries:", num_duplicate_entries)

Number of duplicate entries: 110962


Comment: accident_location, accident, road_surface_cond csv files don't have duplicates, accident_event and person has duplicates

# *Check out atmospheric_cond

In [ ]:
atmo_df = pd.read_csv('ATMOSPHERIC_COND.csv')

atmo_df.head()

,ACCIDENT_NO,ATMOSPH_COND,ATMOSPH_COND_SEQ,ATMOSPH_COND_DESC
0,T20140000502,1,1,Clear
1,T20140000510,1,1,Clear
2,T20140000512,1,1,Clear
3,T20140000522,1,1,Clear
4,T20140000529,1,1,Clear


Comment: it's coorlated to individual incidents and not locations... a bit tricky

# Check out traffic light data
the same method can be used on other datasets

In [117]:
light_df = pd.read_csv('Traffic_Lights_csv.csv')

# Rename columns to uppercase
light_df.columns = light_df.columns.str.upper()


round_lat_lon(light_df, 4)
light_df.head()

,OBJECTID,TLIGHTS_,TLIGHTS_ID,SITE_NO,SITE_NAME,SITE_TYPE,DIRECTORY,DIR_REF,D_ADDED,D_TOWNS,D_EDITED,D_REMOVED,LINK_MODE,STATUS,COMMENTS,MULTI,UFI,ARC_UFI,LATITUDE,LONGITUDE
0,45042,NaN,NaN,5168,BARWON HEADS ROAD/WARALILY BVD,INT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATIONAL,NaN,NaN,NaN,NaN,-38.2367,144.3779
1,45043,NaN,NaN,2015,MELBOURNE NR HOBART (KNOX CITY),POS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATIONAL,NaN,NaN,NaN,NaN,-37.8692,145.2420
2,45044,NaN,NaN,6273,MITCHELL/QUEEN,INT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATIONAL,NaN,NaN,NaN,NaN,-36.7613,144.2803
3,45045,NaN,NaN,520,MOOROODUC/SAGES,INT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATIONAL,Bluetooth,NaN,NaN,NaN,-38.1957,145.1337
4,45046,NaN,NaN,3629,CHURCHILL NR BRAYBROOK COMMUNITY CENTRE,POS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPERATIONAL,NaN,NaN,NaN,NaN,-37.7861,144.8534


In [104]:
merged_df = pd.merge(light_df, node_df, on=['LATITUDE', 'LONGITUDE'], how='inner')
print(len(light_df), len(node_df), len(merged_df))
num_same_coordinates = len(merged_df)
print("Number of times LATITUDE and LONGITUDE are the same in light_df and node_df:", num_same_coordinates)

4852 173707 17137
Number of times LATITUDE and LONGITUDE are the same in light_df and node_df: 17137


In [107]:
# Extract latitude, longitude, and site_no columns from light_df
site_df = light_df[['LATITUDE', 'LONGITUDE', 'SITE_NO']]

# Merge light_data with node_df based on latitude and longitude
merged_df = pd.merge(node_df, site_df, on=['LATITUDE', 'LONGITUDE'], how='left')

# Display the updated node_df DataFrame
merged_df


,ACCIDENT_NO,NODE_ID,NODE_TYPE,AMG_X,AMG_Y,LGA_NAME,LGA NAME ALL,DEG_URBAN_NAME,LATITUDE,LONGITUDE,POSTCODE_CRASH,NUM_INCIDENTS,SITE_NO
0,T20120000009,249102,N,2563628.962,2362700.434,BAW BAW,BAW BAW,RURAL_VICTORIA,-38.2350,145.7267,3981,1,NaN
1,T20120000012,41780,N,2514194.683,2399718.586,MONASH,MONASH,MELB_URBAN,-37.9035,145.1614,3170,14,429.0
2,T20120000013,69811,I,2508689.410,2396130.003,KINGSTON,KINGSTON,MELB_URBAN,-37.9359,145.0988,3169,1,NaN
3,T20120000018,22636,I,2227360.371,2804744.001,MILDURA,MILDURA,RURAL_VICTORIA,-34.2159,142.0439,3505,2,NaN
4,T20120000021,248597,N,2480590.789,2347977.747,MORNINGTON PENINSULA,MORNINGTON PENINSULA,MELB_URBAN,-38.3696,144.7779,3942,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173755,T20240018743,806726,N,2440782.525,2372189.674,GEELONG,GEELONG,LARGE_PROVINCIAL_CITIES,-38.1498,144.3244,3220,1,NaN
173756,T20240019124,1195,I,2442244.883,2366074.321,GEELONG,GEELONG,LARGE_PROVINCIAL_CITIES,-38.2049,144.3406,3216,13,NaN
173757,T20240019328,64871,I,2510793.924,2412236.297,MANNINGHAM,MANNINGHAM,MELB_URBAN,-37.7908,145.1226,3108,1,NaN
173758,T20240019511,807017,N,2496840.489,2407905.752,PORT PHILLIP,PORT PHILLIP,MELB_URBAN,-37.8299,144.9641,3205,1,NaN


In [108]:
merged_df = merged_df[['NODE_ID', 'SITE_NO', 'LATITUDE', 'LONGITUDE', 'POSTCODE_CRASH', 'NUM_INCIDENTS']]
merged_df.to_csv('location_link.csv', index=False)
merged_df

,NODE_ID,SITE_NO,LATITUDE,LONGITUDE,POSTCODE_CRASH,NUM_INCIDENTS
0,249102,NaN,-38.2350,145.7267,3981,1
1,41780,429.0,-37.9035,145.1614,3170,14
2,69811,NaN,-37.9359,145.0988,3169,1
3,22636,NaN,-34.2159,142.0439,3505,2
4,248597,NaN,-38.3696,144.7779,3942,1
...,...,...,...,...,...,...
173755,806726,NaN,-38.1498,144.3244,3220,1
173756,1195,NaN,-38.2049,144.3406,3216,13
173757,64871,NaN,-37.7908,145.1226,3108,1
173758,807017,NaN,-37.8299,144.9641,3205,1


In [111]:
num_entries = merged_df['SITE_NO'].notnull().sum()
print("Number of entries where SITE_NO is not null:", num_entries)

Number of entries where SITE_NO is not null: 17137


# Add Speed sign location

In [119]:
# Can be used for other datasets
sign_df = pd.read_csv('Traffic_Count_Locations_csv.csv')

# Rename columns to uppercase
sign_df.columns = sign_df.columns.str.upper()
sign_df.rename(columns={'ID': 'SITE_NO'}, inplace=True)

round_lat_lon(sign_df, 4)
sign_df.head()

,OBJECTID,TFM_ID,TFM_LRS_ID,TFM_DESC,TFM_TYP_DE,MOVEMENT_T,SITE_DESC,ROAD_NBR,DECLARED_R,LOCAL_ROAD,...,PER_TRUCKS,HF_NUMBER,HLINK_NUMB,HF_ROUTE,HF_ROUTE_D,HF_DESCRIP,HF_SEQUENC,RAMP_TYPE,LATITUDE,LONGITUDE
0,1,14304,36573,PUNT RD N OF ROWENA PDE,INTERSECTION,All Moves,PUNT RD/ROWENA PDE,2080,HODDLE HIGHWAY,PUNT ROAD,...,NaN,218,1514,2,1.0,PUNT RD S BD BTWN BRIDGE RD & BRUNTON AVE,13.0,ROAD,-37.8198,144.9896
1,2,14912,36606,BLACKBURN RD N OF ANDERSONS CREEK RD,INTERSECTION,All Moves,BLACKBURN RD/ANDERSONS CREEK RD,5307,BLACKBURN ROAD,BLACKBURN ROAD,...,NaN,2732,3938,312,1.0,BLACKBURN RD S BD BTWN KING ST & ANDERSONS CRE...,1.0,ROAD,-37.7770,145.1641
2,3,58602,36510,BENDIGO-MARYBOROUGH RD SW BD BTW MARYBOROUGH-D...,MIDBLOCK,All Moves,BENDIGO-MARYBOROUGH RD NBTW MARYBOROUGH-DUNOLL...,5200,BENDIGO-MARYBOROUGH ROAD,MARYBOROUGH-DUNOLLY ROAD,...,0.16,9913,3580,498,1.0,MARYBOROUGH-BENDIGO ROAD S Bd btwn MARYBOROUGH...,7.0,ROAD,-36.9956,143.7627
3,4,860,36609,DORSET_RD N of CANTERBURY_RD,INTERSECTION,Left Missing,DORSET RD/CANTERBURY RD,5784,DORSET ROAD,DORSET ROAD,...,NaN,3520,5282,388,1.0,DORSET RD S BD BTWN EASTFIELD RD & CANTERBURY RD,7.0,ROAD,-37.8256,145.2876
4,5,2565,36543,SYDNEY RD N OF JUKES RD,INTERSECTION,All Moves,SYDNEY RD/JUKES RD,5500,COBURG-CRAIGIEBURN ROAD,SYDNEY ROAD,...,NaN,12817,6626,1620,1.0,SYDNEY RD S BD BTWN METROPOLITAN RING RD & BOX...,93.0,ROAD,-37.7007,144.9601


In [120]:
# Extract latitude, longitude, and site_no columns from light_df
site_df = sign_df[['LATITUDE', 'LONGITUDE', 'SITE_NO']]

# Merge light_data with node_df based on latitude and longitude
merged_df = pd.merge(merged_df, site_df, on=['LATITUDE', 'LONGITUDE'], how='left')

# Display the updated node_df DataFrame
merged_df


KeyError: "['SITE_NO'] not in index"

In [114]:
merged_df = merged_df[['NODE_ID', 'SITE_NO', 'LATITUDE', 'LONGITUDE', 'POSTCODE_CRASH', 'NUM_INCIDENTS']]
merged_df.to_csv('location_link.csv', index=False)
merged_df

,NODE_ID,SITE_NO,LATITUDE,LONGITUDE,POSTCODE_CRASH,NUM_INCIDENTS
0,249102,NaN,-38.2350,145.7267,3981,1
1,41780,429.0,-37.9035,145.1614,3170,14
2,69811,NaN,-37.9359,145.0988,3169,1
3,22636,NaN,-34.2159,142.0439,3505,2
4,248597,NaN,-38.3696,144.7779,3942,1
...,...,...,...,...,...,...
173755,806726,NaN,-38.1498,144.3244,3220,1
173756,1195,NaN,-38.2049,144.3406,3216,13
173757,64871,NaN,-37.7908,145.1226,3108,1
173758,807017,NaN,-37.8299,144.9641,3205,1


In [115]:
num_entries = merged_df['SITE_NO'].notnull().sum()
print("Number of entries where SITE_NO is not null:", num_entries)

num_entries = merged_df['NODE_ID'].notnull().sum()
print("Number of entries where NODE_ID is not null:", num_entries)

Number of entries where SITE_NO is not null: 17137
Number of entries where NODE_ID is not null: 173760


# Mapping things

In [123]:
import geopandas as gpd
import folium

df = pd.read_csv('combined.csv')



C:\Users\Natas\AppData\Local\Temp\ipykernel_41392\3716984904.py:4: DtypeWarning: Columns (9,10,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('combined.csv')


In [126]:
# Check for NaN values and filter them out
df_clean = df.dropna(subset=['LATITUDE', 'LONGITUDE'])

# Create a Folium map centered on the area of interest
# You can use the mean of the coordinates to center the map
center = [df['LATITUDE'].mean(), df['LONGITUDE'].mean()]
m = folium.Map(location=center, zoom_start=5)

# Add markers to the map for each coordinate
for _, row in df.iterrows():
    folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        popup=row['ACCIDENT_NO']
    ).add_to(m)

# Save the map to an HTML file
#m.save('map.html')

# Display the map in a Jupyter Notebook (optional)
m


MemoryError: Unable to allocate 1.42 GiB for an array with shape (22, 8683701) and data type object